In [1]:
# 导入包
import pandas as pd
import gc

In [2]:
# 小样本
# user_log = pd.read_csv('./data_format1/user_log_format1.csv', dtype={'time_stamp':'str'}, nrows = 1000)
# 加载全量样本
user_log = pd.read_csv('./data_format1/user_log_format1.csv', dtype={'time_stamp':'str'})
# 格式化
user_log['user_id'] = user_log['user_id'].astype(str)
# 使用merchant_id（原列名seller_id）
user_log.rename(columns={'seller_id':'merchant_id'}, inplace=True)
user_log['merchant_id'] = user_log['merchant_id'].astype(str)
user_log['item_id'] = user_log['item_id'].astype(str)
user_log['cat_id'] = user_log['cat_id'].astype(str)
user_log['brand_id'].fillna(0, inplace=True)
user_log['brand_id'] = user_log['brand_id'].astype(str)
user_log['time_stamp'] = pd.to_datetime(user_log['time_stamp'], format='%m%d')
user_log['month'] = user_log['time_stamp'].astype(str).str[5:7]
user_log


,user_id,item_id,cat_id,merchant_id,brand_id,time_stamp,action_type,month
0,328862,323294,833,2882,2661.0,1900-08-29,0,08
1,328862,844400,1271,2882,2661.0,1900-08-29,0,08
2,328862,575153,1271,2882,2661.0,1900-08-29,0,08
3,328862,996875,1271,2882,2661.0,1900-08-29,0,08
4,328862,1086186,1271,1253,1049.0,1900-08-29,0,08
...,...,...,...,...,...,...,...,...
54925325,208016,107662,898,1346,7995.0,1900-11-10,0,11
54925326,208016,1058313,898,1346,7995.0,1900-11-10,0,11
54925327,208016,449814,898,983,7995.0,1900-11-10,0,11
54925328,208016,634856,898,1346,7995.0,1900-11-10,0,11


# 数据预处理

In [3]:
user_info = pd.read_csv('./data_format1/user_info_format1.csv')
train_data1 = pd.read_csv('./data_format1/train_format1.csv')
submission = pd.read_csv('./data_format1/test_format1.csv')
train_data = pd.read_csv('./data_format2/train_format2.csv')
train_data1['origin'] = 'train'
submission['origin'] = 'test'
matrix = pd.concat([train_data1, submission], ignore_index=True, sort=False)
matrix.drop(['prob'], axis=1, inplace=True)
# 连接user_info表，通过user_id关联
matrix = matrix.merge(user_info, on='user_id', how='left')
matrix

,user_id,merchant_id,label,origin,age_range,gender
0,34176,3906,0.0,train,6.0,0.0
1,34176,121,0.0,train,6.0,0.0
2,34176,4356,1.0,train,6.0,0.0
3,34176,2217,0.0,train,6.0,0.0
4,230784,4818,0.0,train,0.0,0.0
...,...,...,...,...,...,...
522336,228479,3111,NaN,test,6.0,0.0
522337,97919,2341,NaN,test,8.0,1.0
522338,97919,3971,NaN,test,8.0,1.0
522339,32639,3536,NaN,test,0.0,0.0


In [4]:
matrix['age_range'].fillna(0, inplace=True)
# 0:female, 1:male, 2:unknown
matrix['gender'].fillna(2, inplace=True)
matrix['age_range'] = matrix['age_range'].astype('int8')
matrix['gender'] = matrix['gender'].astype(str)
matrix['label'] = matrix['label'].astype(str)
matrix['user_id'] = matrix['user_id'].astype(str)
matrix['merchant_id'] = matrix['merchant_id'].astype(str)
del user_info, train_data1
gc.collect()
matrix

,user_id,merchant_id,label,origin,age_range,gender
0,34176,3906,0.0,train,6,0.0
1,34176,121,0.0,train,6,0.0
2,34176,4356,1.0,train,6,0.0
3,34176,2217,0.0,train,6,0.0
4,230784,4818,0.0,train,0,0.0
...,...,...,...,...,...,...
522336,228479,3111,nan,test,6,0.0
522337,97919,2341,nan,test,8,1.0
522338,97919,3971,nan,test,8,1.0
522339,32639,3536,nan,test,0,0.0


# 特征处理

In [5]:
train_data['merchant_id'] = train_data['merchant_id'].astype(str)

In [6]:
# User特征处理
user_groups = user_log.groupby(['user_id'])
# 用户交互行为数量 u1
temp = user_groups.size().reset_index().rename(columns={0:'u1'})
matrix = matrix.merge(temp, on='user_id', how='left')
# 使用agg 基于列的聚合操作，统计唯一值的个数 item_id, cat_id, merchant_id, brand_id
#temp = groups['item_id', 'cat_id', 'merchant_id', 'brand_id'].nunique().reset_index().rename(columns={'item_id':'u2', 'cat_id':'u3', 'merchant_id':'u4', 'brand_id':'u5'})
temp = user_groups['item_id'].agg([('u2', 'nunique')]).reset_index()
matrix = matrix.merge(temp, on='user_id', how='left')
temp = user_groups['cat_id'].agg([('u3', 'nunique')]).reset_index()
matrix = matrix.merge(temp, on='user_id', how='left')
temp = user_groups['merchant_id'].agg([('u4', 'nunique')]).reset_index()
matrix = matrix.merge(temp, on='user_id', how='left')
temp = user_groups['brand_id'].agg([('u5', 'nunique')]).reset_index()
matrix = matrix.merge(temp, on='user_id', how='left')

# 时间间隔特征 u6 按照小时
temp = user_groups['time_stamp'].agg([('F_time', 'min'), ('L_time', 'max')]).reset_index()
temp['u6'] = (temp['L_time'] - temp['F_time']).dt.days
matrix = matrix.merge(temp[['user_id', 'u6']], on='user_id', how='left')
# 统计操作类型为0，1，2，3的个数
temp = user_groups['action_type'].value_counts().unstack().reset_index().rename(columns={0:'u7', 1:'u8', 2:'u9', 3:'u10'})
matrix = matrix.merge(temp, on='user_id', how='left')
matrix

,user_id,merchant_id,label,origin,age_range,gender,u1,u2,u3,u4,u5,u6,u7,u8,u9,u10
0,34176,3906,0.0,train,6,0.0,451,256,45,109,108,174,410.0,NaN,34.0,7.0
1,34176,121,0.0,train,6,0.0,451,256,45,109,108,174,410.0,NaN,34.0,7.0
2,34176,4356,1.0,train,6,0.0,451,256,45,109,108,174,410.0,NaN,34.0,7.0
3,34176,2217,0.0,train,6,0.0,451,256,45,109,108,174,410.0,NaN,34.0,7.0
4,230784,4818,0.0,train,0,0.0,54,31,17,20,19,163,47.0,NaN,7.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
522336,228479,3111,nan,test,6,0.0,2004,1173,71,278,282,184,1770.0,NaN,26.0,208.0
522337,97919,2341,nan,test,8,1.0,55,29,14,17,17,138,46.0,NaN,8.0,1.0
522338,97919,3971,nan,test,8,1.0,55,29,14,17,17,138,46.0,NaN,8.0,1.0
522339,32639,3536,nan,test,0,0.0,72,46,24,33,35,172,62.0,1.0,8.0,1.0


In [7]:
# 商家特征处理
merchant_groups = user_log.groupby(['merchant_id'])
# 商家被交互行为数量 m1
temp = merchant_groups.size().reset_index().rename(columns={0:'m1'})
matrix = matrix.merge(temp, on='merchant_id', how='left')
# 统计商家被交互的user_id, item_id, cat_id, brand_id 唯一值
temp = merchant_groups['user_id', 'item_id', 'cat_id', 'brand_id'].nunique().reset_index().rename(columns={'user_id':'m2', 'item_id':'m3', 'cat_id':'m4', 'brand_id':'m5'})
matrix = matrix.merge(temp, on='merchant_id', how='left')
# 统计商家被交互的action_type 唯一值
temp = merchant_groups['action_type'].value_counts().unstack().reset_index().rename(columns={0:'m6', 1:'m7', 2:'m8', 3:'m9'})
matrix = matrix.merge(temp, on='merchant_id', how='left')
# 按照merchant_id 统计随机负采样的个数
temp = train_data[train_data['label']==-1].groupby(['merchant_id']).size().reset_index().rename(columns={0:'m10'})
matrix = matrix.merge(temp, on='merchant_id', how='left')
matrix

,user_id,merchant_id,label,origin,age_range,gender,u1,u2,u3,u4,...,m1,m2,m3,m4,m5,m6,m7,m8,m9,m10
0,34176,3906,0.0,train,6,0.0,451,256,45,109,...,16269,5819,308,20,2,14870.0,28.0,410.0,961.0,2861
1,34176,121,0.0,train,6,0.0,451,256,45,109,...,79865,10931,1179,26,2,72265.0,121.0,4780.0,2699.0,4530
2,34176,4356,1.0,train,6,0.0,451,256,45,109,...,7269,2281,67,15,2,6094.0,16.0,963.0,196.0,1088
3,34176,2217,0.0,train,6,0.0,451,256,45,109,...,60202,16870,377,5,2,52230.0,101.0,3721.0,4150.0,7268
4,230784,4818,0.0,train,0,0.0,54,31,17,20,...,48089,7500,461,27,2,43268.0,129.0,2733.0,1959.0,3102
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
522336,228479,3111,nan,test,6,0.0,2004,1173,71,278,...,10105,4154,542,50,18,8997.0,9.0,687.0,412.0,1982
522337,97919,2341,nan,test,8,1.0,55,29,14,17,...,5543,1592,352,93,19,4548.0,6.0,815.0,174.0,703
522338,97919,3971,nan,test,8,1.0,55,29,14,17,...,28892,7587,272,7,2,24602.0,94.0,2608.0,1588.0,3050
522339,32639,3536,nan,test,0,0.0,72,46,24,33,...,14027,4956,322,19,3,12807.0,29.0,793.0,398.0,2177


In [8]:
# 按照user_id, merchant_id分组
groups = user_log.groupby(['user_id', 'merchant_id'])
temp = groups.size().reset_index().rename(columns={0:'um1'}) #统计行为个数
matrix = matrix.merge(temp, on=['user_id', 'merchant_id'], how='left')
temp = groups['item_id', 'cat_id', 'brand_id'].nunique().reset_index().rename(columns={'item_id':'um2', 'cat_id':'um3', 'brand_id':'um4'}) 
#统计item_id, cat_id, brand_id唯一个数
matrix = matrix.merge(temp, on=['user_id', 'merchant_id'], how='left')
temp = groups['action_type'].value_counts().unstack().reset_index().rename(columns={0:'um5', 1:'um6', 2:'um7', 3:'um8'})#统计不同action_type唯一个数
matrix = matrix.merge(temp, on=['user_id', 'merchant_id'], how='left')
temp = groups['time_stamp'].agg([('first', 'min'), ('last', 'max')]).reset_index()
temp['um9'] = (temp['last'] - temp['first']).dt.days
temp.drop(['first', 'last'], axis=1, inplace=True)
matrix = matrix.merge(temp, on=['user_id', 'merchant_id'], how='left') #统计时间间隔
del temp
matrix

,user_id,merchant_id,label,origin,age_range,gender,u1,u2,u3,u4,...,m10,um1,um2,um3,um4,um5,um6,um7,um8,um9
0,34176,3906,0.0,train,6,0.0,451,256,45,109,...,2861,39,20,6,1,36.0,NaN,1.0,2.0,22
1,34176,121,0.0,train,6,0.0,451,256,45,109,...,4530,14,1,1,1,13.0,NaN,1.0,NaN,3
2,34176,4356,1.0,train,6,0.0,451,256,45,109,...,1088,18,2,1,1,12.0,NaN,6.0,NaN,1
3,34176,2217,0.0,train,6,0.0,451,256,45,109,...,7268,2,1,1,1,1.0,NaN,1.0,NaN,0
4,230784,4818,0.0,train,0,0.0,54,31,17,20,...,3102,8,1,1,1,7.0,NaN,1.0,NaN,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
522336,228479,3111,nan,test,6,0.0,2004,1173,71,278,...,1982,5,2,1,1,4.0,NaN,1.0,NaN,1
522337,97919,2341,nan,test,8,1.0,55,29,14,17,...,703,2,1,1,1,1.0,NaN,1.0,NaN,0
522338,97919,3971,nan,test,8,1.0,55,29,14,17,...,3050,16,5,2,1,12.0,NaN,4.0,NaN,9
522339,32639,3536,nan,test,0,0.0,72,46,24,33,...,2177,3,2,1,1,2.0,NaN,1.0,NaN,0


In [9]:
#用户购买点击比
matrix['r1'] = matrix['u9']/matrix['u7'] 
#商家购买点击比 不需要这个特征值，得分0.68
# matrix['r2'] = matrix['m8']/matrix['m6'] 
#不同用户不同商家购买点击比
matrix['r3'] = matrix['um7']/matrix['um5']
matrix.fillna(0, inplace=True)
# # 修改age_range字段名称为 age_0, age_1, age_2... age_8
temp = pd.get_dummies(matrix['age_range'], prefix='age')
matrix = pd.concat([matrix, temp], axis=1)
temp = pd.get_dummies(matrix['gender'], prefix='g')
matrix = pd.concat([matrix, temp], axis=1)
matrix.drop(['age_range', 'gender'], axis=1, inplace=True)
matrix

,user_id,merchant_id,label,origin,u1,u2,u3,u4,u5,u6,...,age_2,age_3,age_4,age_5,age_6,age_7,age_8,g_0.0,g_1.0,g_2.0
0,34176,3906,0.0,train,451,256,45,109,108,174,...,0,0,0,0,1,0,0,1,0,0
1,34176,121,0.0,train,451,256,45,109,108,174,...,0,0,0,0,1,0,0,1,0,0
2,34176,4356,1.0,train,451,256,45,109,108,174,...,0,0,0,0,1,0,0,1,0,0
3,34176,2217,0.0,train,451,256,45,109,108,174,...,0,0,0,0,1,0,0,1,0,0
4,230784,4818,0.0,train,54,31,17,20,19,163,...,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
522336,228479,3111,nan,test,2004,1173,71,278,282,184,...,0,0,0,0,1,0,0,1,0,0
522337,97919,2341,nan,test,55,29,14,17,17,138,...,0,0,0,0,0,0,1,0,1,0
522338,97919,3971,nan,test,55,29,14,17,17,138,...,0,0,0,0,0,0,1,0,1,0
522339,32639,3536,nan,test,72,46,24,33,35,172,...,0,0,0,0,0,0,0,1,0,0


In [10]:
actions_at_nov = user_log[user_log['month'] == "11"]
buy_actions_at_nov = actions_at_nov[actions_at_nov['action_type'] == 2]

actions_at_oct = user_log.loc[user_log['month'] == "10"]
buy_actions_at_oct = actions_at_oct[actions_at_oct['action_type'] == 2]

actions_at_sep = user_log[user_log['month'] == "09"]
buy_actions_at_sep = actions_at_sep[actions_at_sep['action_type'] == 2]

actions_at_aug = user_log.loc[user_log['month'] == "08"]
buy_actions_at_aug = actions_at_aug[actions_at_aug['action_type'] == 2]

actions_at_jul = user_log[user_log['month'] == "07"]
buy_actions_at_jul = actions_at_jul[actions_at_jul['action_type'] == 2]

actions_at_jun = user_log.loc[user_log['month'] == "06"]
buy_actions_at_jun = actions_at_jun[actions_at_jun['action_type'] == 2]

actions_at_may = user_log.loc[user_log['month'] == "05"]
buy_actions_at_may = actions_at_may[actions_at_may['action_type'] == 2]


user_actions_at_nov = buy_actions_at_nov.groupby(['user_id', 'month'], as_index=False)['month'].agg({'user_buy_cnt_11': 'count'}).fillna(0)
del user_actions_at_nov['month']
user_actions_at_oct = buy_actions_at_oct.groupby(['user_id', 'month'], as_index=False)['month'].agg({'user_buy_cnt_10': 'count'}).fillna(0)
del user_actions_at_oct['month']
user_actions_at_sep = buy_actions_at_sep.groupby(['user_id', 'month'], as_index=False)['month'].agg({'cnt_09': 'count'}).fillna(0)
del user_actions_at_sep['month']
user_actions_at_aug = buy_actions_at_aug.groupby(['user_id', 'month'], as_index=False)['month'].agg({'cnt_08': 'count'}).fillna(0)
del user_actions_at_aug['month']
user_actions_at_jul = buy_actions_at_jul.groupby(['user_id', 'month'], as_index=False)['month'].agg({'cnt_07': 'count'}).fillna(0)
del user_actions_at_jul['month']
user_actions_at_jun = buy_actions_at_jun.groupby(['user_id', 'month'], as_index=False)['month'].agg({'cnt_06': 'count'}).fillna(0)
del user_actions_at_jun['month']
user_actions_at_may = buy_actions_at_may.groupby(['user_id', 'month'], as_index=False)['month'].agg({'cnt_05': 'count'}).fillna(0)
del user_actions_at_may['month']

buy_actions_at_nov.head(5)

,user_id,item_id,cat_id,merchant_id,brand_id,time_stamp,action_type,month
174,328862,406349,1280,2700,5476.0,1900-11-11,2,11
429,234512,240182,81,3018,4144.0,1900-11-11,2,11
430,234512,137298,1432,3271,6957.0,1900-11-11,2,11
432,234512,179830,1208,546,2276.0,1900-11-11,2,11
460,234512,585039,825,800,6326.0,1900-11-10,2,11


In [11]:
res = pd.merge(user_actions_at_nov,user_actions_at_oct, on=['user_id'],how='left')
res = res.merge(user_actions_at_sep, on=['user_id'], how='left')
res = res.merge(user_actions_at_aug, on=['user_id'], how='left')
res = res.merge(user_actions_at_jul, on=['user_id'], how='left')
res = res.merge(user_actions_at_jun, on=['user_id'], how='left')
res = res.merge(user_actions_at_may, on=['user_id'], how='left')
# res['user_buy_deviation_between_months'] = abs(res['cnt'] - res['mean_of_num_of_actions_not_in_nov'])
res = res.fillna(0)
res

,user_id,user_buy_cnt_11,user_buy_cnt_10,cnt_09,cnt_08,cnt_07,cnt_06,cnt_05
0,1,4,2.0,0.0,0.0,0.0,0.0,0.0
1,10,2,5.0,0.0,0.0,0.0,0.0,0.0
2,100,5,3.0,2.0,0.0,0.0,0.0,0.0
3,1000,7,0.0,0.0,0.0,0.0,0.0,0.0
4,10000,6,2.0,0.0,2.0,2.0,3.0,2.0
...,...,...,...,...,...,...,...,...
424165,99995,5,0.0,0.0,0.0,0.0,0.0,0.0
424166,99996,1,0.0,1.0,0.0,0.0,0.0,1.0
424167,99997,1,0.0,0.0,0.0,0.0,0.0,0.0
424168,99998,6,1.0,2.0,0.0,0.0,0.0,0.0


In [12]:
temp=[] 
temp=pd.DataFrame(temp)
temp['user_id'] = res['user_id']
temp['user_buy_mean_between_months'] = res.mean(axis=1)
temp['user_buy_std_between_months'] = res.std(axis=1)
temp['user_buy_min_between_months'] = res.min(axis=1)
temp['user_buy_max_between_months'] = res.max(axis=1)
temp['user_buy_sum_between_months'] = res.sum(axis=1)
temp

,user_id,user_buy_mean_between_months,user_buy_std_between_months,user_buy_min_between_months,user_buy_max_between_months,user_buy_sum_between_months
0,1,0.857143,1.573592,0.0,4.0,6.0
1,10,1.000000,1.914854,0.0,5.0,7.0
2,100,1.428571,1.988060,0.0,5.0,10.0
3,1000,1.000000,2.645751,0.0,7.0,7.0
4,10000,2.428571,1.812654,0.0,6.0,17.0
...,...,...,...,...,...,...
424165,99995,0.714286,1.889822,0.0,5.0,5.0
424166,99996,0.428571,0.534522,0.0,1.0,3.0
424167,99997,0.142857,0.377964,0.0,1.0,1.0
424168,99998,1.285714,2.214670,0.0,6.0,9.0


In [13]:
del res['cnt_09'],res['cnt_08'],res['cnt_07'],res['cnt_06'],res['cnt_05']

In [14]:
matrix = matrix.merge(res, on='user_id', how='left').merge(temp,on='user_id',how='left')
matrix
del res

In [15]:
# action ratio
matrix = matrix.fillna(0)
matrix['user_buy_cnt_11_ratio'] = matrix['user_buy_cnt_11'] / matrix['u9']
matrix['user_buy_cnt_10_ratio'] = matrix['user_buy_cnt_10'] / matrix['u9']
matrix = matrix.fillna(0)
matrix

,user_id,merchant_id,label,origin,u1,u2,u3,u4,u5,u6,...,g_2.0,user_buy_cnt_11,user_buy_cnt_10,user_buy_mean_between_months,user_buy_std_between_months,user_buy_min_between_months,user_buy_max_between_months,user_buy_sum_between_months,user_buy_cnt_11_ratio,user_buy_cnt_10_ratio
0,34176,3906,0.0,train,451,256,45,109,108,174,...,0,15,6.0,4.857143,4.879500,1.0,15.0,34.0,0.441176,0.176471
1,34176,121,0.0,train,451,256,45,109,108,174,...,0,15,6.0,4.857143,4.879500,1.0,15.0,34.0,0.441176,0.176471
2,34176,4356,1.0,train,451,256,45,109,108,174,...,0,15,6.0,4.857143,4.879500,1.0,15.0,34.0,0.441176,0.176471
3,34176,2217,0.0,train,451,256,45,109,108,174,...,0,15,6.0,4.857143,4.879500,1.0,15.0,34.0,0.441176,0.176471
4,230784,4818,0.0,train,54,31,17,20,19,163,...,0,1,1.0,1.000000,1.414214,0.0,4.0,7.0,0.142857,0.142857
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
522336,228479,3111,nan,test,2004,1173,71,278,282,184,...,0,3,8.0,3.714286,2.288689,1.0,8.0,26.0,0.115385,0.307692
522337,97919,2341,nan,test,55,29,14,17,17,138,...,0,5,0.0,1.142857,1.864454,0.0,5.0,8.0,0.625000,0.000000
522338,97919,3971,nan,test,55,29,14,17,17,138,...,0,5,0.0,1.142857,1.864454,0.0,5.0,8.0,0.625000,0.000000
522339,32639,3536,nan,test,72,46,24,33,35,172,...,0,2,2.0,1.142857,0.690066,0.0,2.0,8.0,0.250000,0.250000


In [16]:
mer_actions_at_nov = buy_actions_at_nov.groupby(['merchant_id', 'month'], as_index=False)['month'].agg({'m_cnt_11': 'count'}).fillna(0)
del mer_actions_at_nov['month']
mer_actions_at_oct = buy_actions_at_oct.groupby(['merchant_id', 'month'], as_index=False)['month'].agg({'m_cnt_10': 'count'}).fillna(0)
del mer_actions_at_oct['month']
mer_actions_at_sep = buy_actions_at_sep.groupby(['merchant_id', 'month'], as_index=False)['month'].agg({'cnt_09': 'count'}).fillna(0)
del mer_actions_at_sep['month']
mer_actions_at_aug = buy_actions_at_aug.groupby(['merchant_id', 'month'], as_index=False)['month'].agg({'cnt_08': 'count'}).fillna(0)
del mer_actions_at_aug['month']
mer_actions_at_jul = buy_actions_at_jul.groupby(['merchant_id', 'month'], as_index=False)['month'].agg({'cnt_07': 'count'}).fillna(0)
del mer_actions_at_jul['month']
mer_actions_at_jun = buy_actions_at_jun.groupby(['merchant_id', 'month'], as_index=False)['month'].agg({'cnt_06': 'count'}).fillna(0)
del mer_actions_at_jun['month']
mer_actions_at_may = buy_actions_at_may.groupby(['merchant_id', 'month'], as_index=False)['month'].agg({'cnt_05': 'count'}).fillna(0)
del mer_actions_at_may['month']

In [17]:
res1 = pd.merge(mer_actions_at_nov,mer_actions_at_oct, on=['merchant_id'], how='left')
res1 = res1.merge(mer_actions_at_sep, on=['merchant_id'], how='left')
res1 = res1.merge(mer_actions_at_aug, on=['merchant_id'], how='left')
res1 = res1.merge(mer_actions_at_jul, on=['merchant_id'], how='left')
res1 = res1.merge(mer_actions_at_jun, on=['merchant_id'], how='left')
res1 = res1.merge(mer_actions_at_may, on=['merchant_id'], how='left')
res1 = res1.fillna(0)
# res['user_buy_deviation_between_months'] = abs(res['cnt'] - res['mean_of_num_of_actions_not_in_nov'])
temp=[] 
temp=pd.DataFrame(temp)
temp['merchant_id'] = res1['merchant_id']
temp['merchant_buy_mean_between_months'] = res1.mean(axis=1)
temp['merchant_buy_std_between_months'] = res1.std(axis=1)
temp['merchant_buy_min_between_months'] = res1.min(axis=1)
temp['merchant_buy_max_between_months'] = res1.max(axis=1)
temp['merchant_buy_sum_between_months'] = res1.sum(axis=1)
temp
# res = res.loc[:, ['user_id', 'merchant_id', 'user_buy_deviation_between_nov_and_before']]

,merchant_id,merchant_buy_mean_between_months,merchant_buy_std_between_months,merchant_buy_min_between_months,merchant_buy_max_between_months,merchant_buy_sum_between_months
0,1,2529.285714,2268.896921,1320.0,7594.0,17705.0
1,10,161.857143,368.508900,1.0,997.0,1133.0
2,100,76.857143,109.929415,0.0,251.0,538.0
3,1000,137.000000,182.047613,27.0,544.0,959.0
4,1001,28.000000,21.931712,8.0,68.0,196.0
...,...,...,...,...,...,...
4987,995,34.857143,30.295371,6.0,89.0,244.0
4988,996,17.428571,40.057696,0.0,108.0,122.0
4989,997,39.428571,48.822321,3.0,126.0,276.0
4990,998,97.714286,104.172476,0.0,261.0,684.0


In [18]:
del res1['cnt_09'],res1['cnt_08'],res1['cnt_07'],res1['cnt_06'],res1['cnt_05']
matrix = matrix.merge(res1, on='merchant_id', how='left').merge(temp,on='merchant_id',how='left')
matrix
del res1

In [19]:
matrix

,user_id,merchant_id,label,origin,u1,u2,u3,u4,u5,u6,...,user_buy_sum_between_months,user_buy_cnt_11_ratio,user_buy_cnt_10_ratio,m_cnt_11,m_cnt_10,merchant_buy_mean_between_months,merchant_buy_std_between_months,merchant_buy_min_between_months,merchant_buy_max_between_months,merchant_buy_sum_between_months
0,34176,3906,0.0,train,451,256,45,109,108,174,...,34.0,0.441176,0.176471,178,29.0,58.571429,56.314170,14.0,178.0,410.0
1,34176,121,0.0,train,451,256,45,109,108,174,...,34.0,0.441176,0.176471,3299,267.0,682.857143,1159.388119,77.0,3299.0,4780.0
2,34176,4356,1.0,train,451,256,45,109,108,174,...,34.0,0.441176,0.176471,167,107.0,137.571429,127.634709,0.0,391.0,963.0
3,34176,2217,0.0,train,451,256,45,109,108,174,...,34.0,0.441176,0.176471,2715,371.0,531.571429,967.556003,81.0,2715.0,3721.0
4,230784,4818,0.0,train,54,31,17,20,19,163,...,7.0,0.142857,0.142857,2161,102.0,390.428571,780.975215,68.0,2161.0,2733.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
522336,228479,3111,nan,test,2004,1173,71,278,282,184,...,26.0,0.115385,0.307692,191,97.0,98.142857,42.534020,66.0,191.0,687.0
522337,97919,2341,nan,test,55,29,14,17,17,138,...,8.0,0.625000,0.000000,309,87.0,116.428571,89.136332,33.0,309.0,815.0
522338,97919,3971,nan,test,55,29,14,17,17,138,...,8.0,0.625000,0.000000,1767,177.0,372.571429,617.024002,54.0,1767.0,2608.0
522339,32639,3536,nan,test,72,46,24,33,35,172,...,8.0,0.250000,0.250000,753,21.0,113.285714,282.194209,0.0,753.0,793.0


In [20]:
um_actions_at_nov = buy_actions_at_nov.groupby(['user_id','merchant_id', 'month'], as_index=False)['month'].agg({'um_cnt_11': 'count'}).fillna(0)
del um_actions_at_nov['month']
um_actions_at_oct = buy_actions_at_oct.groupby(['user_id','merchant_id', 'month'], as_index=False)['month'].agg({'um_cnt_10': 'count'}).fillna(0)
del um_actions_at_oct['month']
um_actions_at_sep = buy_actions_at_sep.groupby(['user_id','merchant_id', 'month'], as_index=False)['month'].agg({'cnt_09': 'count'}).fillna(0)
del um_actions_at_sep['month']
um_actions_at_aug = buy_actions_at_aug.groupby(['user_id','merchant_id', 'month'], as_index=False)['month'].agg({'cnt_08': 'count'}).fillna(0)
del um_actions_at_aug['month']
um_actions_at_jul = buy_actions_at_jul.groupby(['user_id','merchant_id', 'month'], as_index=False)['month'].agg({'cnt_07': 'count'}).fillna(0)
del um_actions_at_jul['month']
um_actions_at_jun = buy_actions_at_jun.groupby(['user_id','merchant_id', 'month'], as_index=False)['month'].agg({'cnt_06': 'count'}).fillna(0)
del um_actions_at_jun['month']
um_actions_at_may = buy_actions_at_may.groupby(['user_id','merchant_id', 'month'], as_index=False)['month'].agg({'cnt_05': 'count'}).fillna(0)
del um_actions_at_may['month']


In [21]:
res2 = pd.merge(um_actions_at_nov,um_actions_at_oct, on=['user_id','merchant_id'], how='left')
res2 = res2.merge(um_actions_at_sep, on=['user_id','merchant_id'], how='left')
res2 = res2.merge(um_actions_at_aug, on=['user_id','merchant_id'], how='left')
res2 = res2.merge(um_actions_at_jul, on=['user_id','merchant_id'], how='left')
res2 = res2.merge(um_actions_at_jun, on=['user_id','merchant_id'], how='left')
res2 = res2.merge(um_actions_at_may, on=['user_id','merchant_id'], how='left')
# res['user_buy_deviation_between_months'] = abs(res['cnt'] - res['mean_of_num_of_actions_not_in_nov'])
res2 = res2.fillna(0)
temp=[] 
temp=pd.DataFrame(temp)
temp['user_id'] = res2['user_id']
temp['merchant_id'] = res2['merchant_id']
temp['um_buy_mean_between_months'] = res2.mean(axis=1)
temp['um_buy_std_between_months'] = res2.std(axis=1)
temp['um_buy_min_between_months'] = res2.min(axis=1)
temp['um_buy_max_between_months'] = res2.max(axis=1)
temp['um_buy_sum_between_months'] = res2.sum(axis=1)
temp.head(5)

,user_id,merchant_id,um_buy_mean_between_months,um_buy_std_between_months,um_buy_min_between_months,um_buy_max_between_months,um_buy_sum_between_months
0,1,1019,0.571429,1.511858,0.0,4.0,4.0
1,10,938,0.285714,0.755929,0.0,2.0,2.0
2,100,1,0.571429,1.133893,0.0,3.0,4.0
3,100,1461,0.142857,0.377964,0.0,1.0,1.0
4,100,2537,0.285714,0.755929,0.0,2.0,2.0


In [22]:
del res2['cnt_09'],res2['cnt_08'],res2['cnt_07'],res2['cnt_06'],res2['cnt_05']
matrix = matrix.merge(res2, on=['user_id','merchant_id'], how='left').merge(temp,on=['user_id','merchant_id'],how='left')
del res2
matrix

,user_id,merchant_id,label,origin,u1,u2,u3,u4,u5,u6,...,merchant_buy_min_between_months,merchant_buy_max_between_months,merchant_buy_sum_between_months,um_cnt_11,um_cnt_10,um_buy_mean_between_months,um_buy_std_between_months,um_buy_min_between_months,um_buy_max_between_months,um_buy_sum_between_months
0,34176,3906,0.0,train,451,256,45,109,108,174,...,14.0,178.0,410.0,1,0.0,0.142857,0.377964,0.0,1.0,1.0
1,34176,121,0.0,train,451,256,45,109,108,174,...,77.0,3299.0,4780.0,1,0.0,0.142857,0.377964,0.0,1.0,1.0
2,34176,4356,1.0,train,451,256,45,109,108,174,...,0.0,391.0,963.0,6,0.0,0.857143,2.267787,0.0,6.0,6.0
3,34176,2217,0.0,train,451,256,45,109,108,174,...,81.0,2715.0,3721.0,1,0.0,0.142857,0.377964,0.0,1.0,1.0
4,230784,4818,0.0,train,54,31,17,20,19,163,...,68.0,2161.0,2733.0,1,0.0,0.142857,0.377964,0.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
522336,228479,3111,nan,test,2004,1173,71,278,282,184,...,66.0,191.0,687.0,1,0.0,0.142857,0.377964,0.0,1.0,1.0
522337,97919,2341,nan,test,55,29,14,17,17,138,...,33.0,309.0,815.0,1,0.0,0.142857,0.377964,0.0,1.0,1.0
522338,97919,3971,nan,test,55,29,14,17,17,138,...,54.0,1767.0,2608.0,4,0.0,0.571429,1.511858,0.0,4.0,4.0
522339,32639,3536,nan,test,72,46,24,33,35,172,...,0.0,753.0,793.0,1,0.0,0.142857,0.377964,0.0,1.0,1.0


In [23]:
# Repeat buyer features
buy_actions = user_log[user_log['action_type'] == 2]
um_groups = buy_actions.groupby(['merchant_id','user_id'])
temp = um_groups.size().reset_index().rename(columns={0:'mu1'}) #统计行为个数
temp = temp[temp['mu1']>=2]
temp = temp.groupby('merchant_id').size().reset_index().rename(columns={0:'repeat_buyer_count'})
temp

,merchant_id,repeat_buyer_count
0,1,3779
1,10,107
2,100,127
3,1000,76
4,1001,32
...,...,...
4970,995,15
4971,996,15
4972,997,28
4973,998,164


In [24]:
# Repeat buyer features
matrix = matrix.merge(temp, on=['merchant_id'], how='left')
matrix

,user_id,merchant_id,label,origin,u1,u2,u3,u4,u5,u6,...,merchant_buy_max_between_months,merchant_buy_sum_between_months,um_cnt_11,um_cnt_10,um_buy_mean_between_months,um_buy_std_between_months,um_buy_min_between_months,um_buy_max_between_months,um_buy_sum_between_months,repeat_buyer_count
0,34176,3906,0.0,train,451,256,45,109,108,174,...,178.0,410.0,1,0.0,0.142857,0.377964,0.0,1.0,1.0,66.0
1,34176,121,0.0,train,451,256,45,109,108,174,...,3299.0,4780.0,1,0.0,0.142857,0.377964,0.0,1.0,1.0,1001.0
2,34176,4356,1.0,train,451,256,45,109,108,174,...,391.0,963.0,6,0.0,0.857143,2.267787,0.0,6.0,6.0,170.0
3,34176,2217,0.0,train,451,256,45,109,108,174,...,2715.0,3721.0,1,0.0,0.142857,0.377964,0.0,1.0,1.0,316.0
4,230784,4818,0.0,train,54,31,17,20,19,163,...,2161.0,2733.0,1,0.0,0.142857,0.377964,0.0,1.0,1.0,543.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
522336,228479,3111,nan,test,2004,1173,71,278,282,184,...,191.0,687.0,1,0.0,0.142857,0.377964,0.0,1.0,1.0,72.0
522337,97919,2341,nan,test,55,29,14,17,17,138,...,309.0,815.0,1,0.0,0.142857,0.377964,0.0,1.0,1.0,175.0
522338,97919,3971,nan,test,55,29,14,17,17,138,...,1767.0,2608.0,4,0.0,0.571429,1.511858,0.0,4.0,4.0,431.0
522339,32639,3536,nan,test,72,46,24,33,35,172,...,753.0,793.0,1,0.0,0.142857,0.377964,0.0,1.0,1.0,89.0


In [25]:
# 最后一个月 一个星期 一天的购物数量
actions_at_1111 = user_log[user_log['time_stamp'] == "1900-11-11"]
actions_at_lastweek = user_log[user_log['time_stamp'] > '1900-11-04']
actions_at_lastweek

,user_id,item_id,cat_id,merchant_id,brand_id,time_stamp,action_type,month
171,328862,406349,1280,2700,5476.0,1900-11-11,0,11
172,328862,406349,1280,2700,5476.0,1900-11-11,0,11
173,328862,807126,1181,1963,6109.0,1900-11-11,0,11
174,328862,406349,1280,2700,5476.0,1900-11-11,2,11
175,328862,406349,1280,2700,5476.0,1900-11-11,0,11
...,...,...,...,...,...,...,...,...
54925325,208016,107662,898,1346,7995.0,1900-11-10,0,11
54925326,208016,1058313,898,1346,7995.0,1900-11-10,0,11
54925327,208016,449814,898,983,7995.0,1900-11-10,0,11
54925328,208016,634856,898,1346,7995.0,1900-11-10,0,11


In [26]:
# 最后一个星期 一天的购物数量
u_buy_actions_at_1111 = actions_at_1111.groupby(['user_id']).size().reset_index().rename(columns={0:'u1_buy_1111'})
matrix = matrix.merge(u_buy_actions_at_1111, on=['user_id'], how='left')
del u_buy_actions_at_1111
matrix

,user_id,merchant_id,label,origin,u1,u2,u3,u4,u5,u6,...,merchant_buy_sum_between_months,um_cnt_11,um_cnt_10,um_buy_mean_between_months,um_buy_std_between_months,um_buy_min_between_months,um_buy_max_between_months,um_buy_sum_between_months,repeat_buyer_count,u1_buy_1111
0,34176,3906,0.0,train,451,256,45,109,108,174,...,410.0,1,0.0,0.142857,0.377964,0.0,1.0,1.0,66.0,36
1,34176,121,0.0,train,451,256,45,109,108,174,...,4780.0,1,0.0,0.142857,0.377964,0.0,1.0,1.0,1001.0,36
2,34176,4356,1.0,train,451,256,45,109,108,174,...,963.0,6,0.0,0.857143,2.267787,0.0,6.0,6.0,170.0,36
3,34176,2217,0.0,train,451,256,45,109,108,174,...,3721.0,1,0.0,0.142857,0.377964,0.0,1.0,1.0,316.0,36
4,230784,4818,0.0,train,54,31,17,20,19,163,...,2733.0,1,0.0,0.142857,0.377964,0.0,1.0,1.0,543.0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
522336,228479,3111,nan,test,2004,1173,71,278,282,184,...,687.0,1,0.0,0.142857,0.377964,0.0,1.0,1.0,72.0,17
522337,97919,2341,nan,test,55,29,14,17,17,138,...,815.0,1,0.0,0.142857,0.377964,0.0,1.0,1.0,175.0,9
522338,97919,3971,nan,test,55,29,14,17,17,138,...,2608.0,4,0.0,0.571429,1.511858,0.0,4.0,4.0,431.0,9
522339,32639,3536,nan,test,72,46,24,33,35,172,...,793.0,1,0.0,0.142857,0.377964,0.0,1.0,1.0,89.0,30


In [27]:
# Latest one-week features and latest one-month features
u_buy_actions_at_lastweek = actions_at_lastweek.groupby(['user_id']).size().reset_index().rename(columns={0:'u1_buy_last_week'})
# u_buy_actions_at_lastweek
matrix = matrix.merge(u_buy_actions_at_lastweek, on=['user_id'], how='left')
del u_buy_actions_at_lastweek
matrix

,user_id,merchant_id,label,origin,u1,u2,u3,u4,u5,u6,...,um_cnt_11,um_cnt_10,um_buy_mean_between_months,um_buy_std_between_months,um_buy_min_between_months,um_buy_max_between_months,um_buy_sum_between_months,repeat_buyer_count,u1_buy_1111,u1_buy_last_week
0,34176,3906,0.0,train,451,256,45,109,108,174,...,1,0.0,0.142857,0.377964,0.0,1.0,1.0,66.0,36,119
1,34176,121,0.0,train,451,256,45,109,108,174,...,1,0.0,0.142857,0.377964,0.0,1.0,1.0,1001.0,36,119
2,34176,4356,1.0,train,451,256,45,109,108,174,...,6,0.0,0.857143,2.267787,0.0,6.0,6.0,170.0,36,119
3,34176,2217,0.0,train,451,256,45,109,108,174,...,1,0.0,0.142857,0.377964,0.0,1.0,1.0,316.0,36,119
4,230784,4818,0.0,train,54,31,17,20,19,163,...,1,0.0,0.142857,0.377964,0.0,1.0,1.0,543.0,2,12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
522336,228479,3111,nan,test,2004,1173,71,278,282,184,...,1,0.0,0.142857,0.377964,0.0,1.0,1.0,72.0,17,115
522337,97919,2341,nan,test,55,29,14,17,17,138,...,1,0.0,0.142857,0.377964,0.0,1.0,1.0,175.0,9,30
522338,97919,3971,nan,test,55,29,14,17,17,138,...,4,0.0,0.571429,1.511858,0.0,4.0,4.0,431.0,9,30
522339,32639,3536,nan,test,72,46,24,33,35,172,...,1,0.0,0.142857,0.377964,0.0,1.0,1.0,89.0,30,34


In [28]:
matrix = matrix.fillna(0)
matrix['user_buy_cnt_1111_ratio'] = matrix['u1_buy_1111'] / matrix['u9']
matrix['user_buy_cnt_last_week_ratio'] = matrix['u1_buy_last_week'] / matrix['u9']
matrix = matrix.fillna(0)
matrix

,user_id,merchant_id,label,origin,u1,u2,u3,u4,u5,u6,...,um_buy_mean_between_months,um_buy_std_between_months,um_buy_min_between_months,um_buy_max_between_months,um_buy_sum_between_months,repeat_buyer_count,u1_buy_1111,u1_buy_last_week,user_buy_cnt_1111_ratio,user_buy_cnt_last_week_ratio
0,34176,3906,0.0,train,451,256,45,109,108,174,...,0.142857,0.377964,0.0,1.0,1.0,66.0,36,119,1.058824,3.500000
1,34176,121,0.0,train,451,256,45,109,108,174,...,0.142857,0.377964,0.0,1.0,1.0,1001.0,36,119,1.058824,3.500000
2,34176,4356,1.0,train,451,256,45,109,108,174,...,0.857143,2.267787,0.0,6.0,6.0,170.0,36,119,1.058824,3.500000
3,34176,2217,0.0,train,451,256,45,109,108,174,...,0.142857,0.377964,0.0,1.0,1.0,316.0,36,119,1.058824,3.500000
4,230784,4818,0.0,train,54,31,17,20,19,163,...,0.142857,0.377964,0.0,1.0,1.0,543.0,2,12,0.285714,1.714286
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
522336,228479,3111,nan,test,2004,1173,71,278,282,184,...,0.142857,0.377964,0.0,1.0,1.0,72.0,17,115,0.653846,4.423077
522337,97919,2341,nan,test,55,29,14,17,17,138,...,0.142857,0.377964,0.0,1.0,1.0,175.0,9,30,1.125000,3.750000
522338,97919,3971,nan,test,55,29,14,17,17,138,...,0.571429,1.511858,0.0,4.0,4.0,431.0,9,30,1.125000,3.750000
522339,32639,3536,nan,test,72,46,24,33,35,172,...,0.142857,0.377964,0.0,1.0,1.0,89.0,30,34,3.750000,4.250000


In [29]:
# User aggregation features
um_buy_actions_day = buy_actions.groupby(['merchant_id', 'user_id'], as_index=False)['time_stamp'].agg({'um_agg_day': 'count'}).fillna(0)
um_buy_actions_day.head(5)

,merchant_id,user_id,um_agg_day
0,1,100,4
1,1,100099,2
2,1,100136,1
3,1,100196,13
4,1,100257,6


In [30]:
um_buy_actions_day_tmp = um_buy_actions_day.groupby('merchant_id')['um_agg_day'].agg([('um_agg_day_min', 'min'), ('um_agg_day_max', 'max'),('um_agg_day_mean', 'mean'),('um_agg_day_std', 'std'), ('um_agg_day_sum', 'sum')]).reset_index().fillna(0)
um_buy_actions_day_tmp.head(5)
matrix = matrix.merge(um_buy_actions_day_tmp, on=['merchant_id'], how='left')
del um_buy_actions_day,um_buy_actions_day_tmp
matrix

,user_id,merchant_id,label,origin,u1,u2,u3,u4,u5,u6,...,repeat_buyer_count,u1_buy_1111,u1_buy_last_week,user_buy_cnt_1111_ratio,user_buy_cnt_last_week_ratio,um_agg_day_min,um_agg_day_max,um_agg_day_mean,um_agg_day_std,um_agg_day_sum
0,34176,3906,0.0,train,451,256,45,109,108,174,...,66.0,36,119,1.058824,3.500000,1,8,1.322581,0.791831,410
1,34176,121,0.0,train,451,256,45,109,108,174,...,1001.0,36,119,1.058824,3.500000,1,16,1.735657,1.422353,4780
2,34176,4356,1.0,train,451,256,45,109,108,174,...,170.0,36,119,1.058824,3.500000,1,11,1.426667,1.030053,963
3,34176,2217,0.0,train,451,256,45,109,108,174,...,316.0,36,119,1.058824,3.500000,1,17,1.123151,0.502937,3721
4,230784,4818,0.0,train,54,31,17,20,19,163,...,543.0,2,12,0.285714,1.714286,1,15,1.739656,1.438444,2733
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
522336,228479,3111,nan,test,2004,1173,71,278,282,184,...,72.0,17,115,0.653846,4.423077,1,7,1.201049,0.658874,687
522337,97919,2341,nan,test,55,29,14,17,17,138,...,175.0,9,30,1.125000,3.750000,1,17,2.089744,2.029258,815
522338,97919,3971,nan,test,55,29,14,17,17,138,...,431.0,9,30,1.125000,3.750000,1,10,1.306613,0.715161,2608
522339,32639,3536,nan,test,72,46,24,33,35,172,...,89.0,30,34,3.750000,4.250000,1,9,1.199697,0.650920,793


In [31]:
# User aggregation features
um_buy_actions_brand = buy_actions.groupby(['merchant_id', 'user_id'], as_index=False)['brand_id'].agg({'um_agg_brand': 'nunique'}).fillna(0)
um_buy_actions_brand.head(5)

,merchant_id,user_id,um_agg_brand
0,1,100,1
1,1,100099,1
2,1,100136,1
3,1,100196,2
4,1,100257,1


In [32]:
um_buy_actions_brand_tmp = um_buy_actions_brand.groupby('merchant_id')['um_agg_brand'].agg([('um_agg_brand_min', 'min'), ('um_agg_brand_max', 'max'),('um_agg_brand_mean', 'mean'),('um_agg_brand_std', 'std'), ('um_agg_brand_sum', 'sum')]).reset_index().fillna(0)
print(um_buy_actions_brand_tmp.head(5))

matrix = matrix.merge(um_buy_actions_brand_tmp, on=['merchant_id'], how='left')
del um_buy_actions_brand,um_buy_actions_brand_tmp
matrix

  merchant_id  um_agg_brand_min  um_agg_brand_max  um_agg_brand_mean  \
0           1                 1                 2           1.020089   
1          10                 1                 1           1.000000   
2         100                 1                 2           1.049231   
3        1000                 1                 2           1.016279   
4        1001                 1                 1           1.000000   

   um_agg_brand_std  um_agg_brand_sum  
0          0.140313              7820  
1          0.000000               995  
2          0.216683               341  
3          0.126620               874  
4          0.000000               158  


,user_id,merchant_id,label,origin,u1,u2,u3,u4,u5,u6,...,um_agg_day_min,um_agg_day_max,um_agg_day_mean,um_agg_day_std,um_agg_day_sum,um_agg_brand_min,um_agg_brand_max,um_agg_brand_mean,um_agg_brand_std,um_agg_brand_sum
0,34176,3906,0.0,train,451,256,45,109,108,174,...,1,8,1.322581,0.791831,410,1,1,1.000000,0.000000,310
1,34176,121,0.0,train,451,256,45,109,108,174,...,1,16,1.735657,1.422353,4780,1,1,1.000000,0.000000,2754
2,34176,4356,1.0,train,451,256,45,109,108,174,...,1,11,1.426667,1.030053,963,1,1,1.000000,0.000000,675
3,34176,2217,0.0,train,451,256,45,109,108,174,...,1,17,1.123151,0.502937,3721,1,1,1.000000,0.000000,3313
4,230784,4818,0.0,train,54,31,17,20,19,163,...,1,15,1.739656,1.438444,2733,1,1,1.000000,0.000000,1571
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
522336,228479,3111,nan,test,2004,1173,71,278,282,184,...,1,7,1.201049,0.658874,687,1,2,1.026224,0.159940,587
522337,97919,2341,nan,test,55,29,14,17,17,138,...,1,17,2.089744,2.029258,815,1,7,1.412821,0.769829,551
522338,97919,3971,nan,test,55,29,14,17,17,138,...,1,10,1.306613,0.715161,2608,1,1,1.000000,0.000000,1996
522339,32639,3536,nan,test,72,46,24,33,35,172,...,1,9,1.199697,0.650920,793,1,2,1.003026,0.054965,663


In [33]:
# User aggregation features
um_buy_actions_item = buy_actions.groupby(['merchant_id', 'user_id'], as_index=False)['item_id'].agg({'um_agg_item': 'nunique'}).fillna(0)
um_buy_actions_item.head(5)

,merchant_id,user_id,um_agg_item
0,1,100,4
1,1,100099,2
2,1,100136,1
3,1,100196,13
4,1,100257,6


In [34]:
um_buy_actions_item_tmp = um_buy_actions_item.groupby('merchant_id')['um_agg_item'].agg([('um_agg_item_min', 'min'), ('um_agg_item_max', 'max'),('um_agg_item_mean', 'mean'),('um_agg_item_std', 'std'),('um_agg_item_sum', 'sum')]).reset_index().fillna(0)
um_buy_actions_item_tmp.head(5)

matrix = matrix.merge(um_buy_actions_item_tmp, on=['merchant_id'], how='left')
del um_buy_actions_item,um_buy_actions_item_tmp
matrix

,user_id,merchant_id,label,origin,u1,u2,u3,u4,u5,u6,...,um_agg_brand_min,um_agg_brand_max,um_agg_brand_mean,um_agg_brand_std,um_agg_brand_sum,um_agg_item_min,um_agg_item_max,um_agg_item_mean,um_agg_item_std,um_agg_item_sum
0,34176,3906,0.0,train,451,256,45,109,108,174,...,1,1,1.000000,0.000000,310,1,6,1.258065,0.676357,390
1,34176,121,0.0,train,451,256,45,109,108,174,...,1,1,1.000000,0.000000,2754,1,13,1.633987,1.247600,4500
2,34176,4356,1.0,train,451,256,45,109,108,174,...,1,1,1.000000,0.000000,675,1,6,1.149630,0.483998,776
3,34176,2217,0.0,train,451,256,45,109,108,174,...,1,1,1.000000,0.000000,3313,1,9,1.074857,0.354166,3561
4,230784,4818,0.0,train,54,31,17,20,19,163,...,1,1,1.000000,0.000000,1571,1,15,1.661999,1.304944,2611
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
522336,228479,3111,nan,test,2004,1173,71,278,282,184,...,1,2,1.026224,0.159940,587,1,5,1.148601,0.470438,657
522337,97919,2341,nan,test,55,29,14,17,17,138,...,1,7,1.412821,0.769829,551,1,15,1.892308,1.742761,738
522338,97919,3971,nan,test,55,29,14,17,17,138,...,1,1,1.000000,0.000000,1996,1,9,1.276553,0.661639,2548
522339,32639,3536,nan,test,72,46,24,33,35,172,...,1,2,1.003026,0.054965,663,1,5,1.122542,0.417558,742


In [35]:
# Merchant aggregation features-day
mu_buy_actions_day = buy_actions.groupby(['user_id','merchant_id'], as_index=False)['time_stamp'].agg({'mu_agg_day': 'count'}).fillna(0)
print(mu_buy_actions_day.head(5))
mu_buy_actions_day_tmp = mu_buy_actions_day.groupby('user_id')['mu_agg_day'].agg([('mu_agg_day_min', 'min'), ('mu_agg_day_max', 'max'),('mu_agg_day_mean', 'mean'),('mu_agg_day_std', 'std'), ('mu_agg_day_sum', 'sum')]).reset_index().fillna(0)
mu_buy_actions_day_tmp.head(5)

  user_id merchant_id  mu_agg_day
0       1        1019           4
1       1        4026           1
2       1         925           1
3      10        1364           5
4      10         938           2


,user_id,mu_agg_day_min,mu_agg_day_max,mu_agg_day_mean,mu_agg_day_std,mu_agg_day_sum
0,1,1,4,2.000000,1.732051,6
1,10,2,5,3.500000,2.121320,7
2,100,1,4,1.666667,1.211060,10
3,1000,1,3,1.400000,0.894427,7
4,10000,1,2,1.307692,0.480384,17


In [36]:
matrix = matrix.merge(mu_buy_actions_day_tmp, on=['user_id'], how='left')
del mu_buy_actions_day_tmp,mu_buy_actions_day
matrix

,user_id,merchant_id,label,origin,u1,u2,u3,u4,u5,u6,...,um_agg_item_min,um_agg_item_max,um_agg_item_mean,um_agg_item_std,um_agg_item_sum,mu_agg_day_min,mu_agg_day_max,mu_agg_day_mean,mu_agg_day_std,mu_agg_day_sum
0,34176,3906,0.0,train,451,256,45,109,108,174,...,1,6,1.258065,0.676357,390,1,6,1.545455,1.299350,34
1,34176,121,0.0,train,451,256,45,109,108,174,...,1,13,1.633987,1.247600,4500,1,6,1.545455,1.299350,34
2,34176,4356,1.0,train,451,256,45,109,108,174,...,1,6,1.149630,0.483998,776,1,6,1.545455,1.299350,34
3,34176,2217,0.0,train,451,256,45,109,108,174,...,1,9,1.074857,0.354166,3561,1,6,1.545455,1.299350,34
4,230784,4818,0.0,train,54,31,17,20,19,163,...,1,15,1.661999,1.304944,2611,1,2,1.166667,0.408248,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
522336,228479,3111,nan,test,2004,1173,71,278,282,184,...,1,5,1.148601,0.470438,657,1,2,1.238095,0.436436,26
522337,97919,2341,nan,test,55,29,14,17,17,138,...,1,15,1.892308,1.742761,738,1,4,2.000000,1.414214,8
522338,97919,3971,nan,test,55,29,14,17,17,138,...,1,9,1.276553,0.661639,2548,1,4,2.000000,1.414214,8
522339,32639,3536,nan,test,72,46,24,33,35,172,...,1,5,1.122542,0.417558,742,1,1,1.000000,0.000000,8


In [37]:
# Merchant aggregation features-day
mu_buy_actions_brand = buy_actions.groupby(['user_id','merchant_id'], as_index=False)['brand_id'].agg({'mu_agg_brand': 'nunique'}).fillna(0)
print(mu_buy_actions_brand.head(5))
mu_buy_actions_brand_tmp = mu_buy_actions_brand.groupby('user_id')['mu_agg_brand'].agg([('mu_agg_brand_min', 'min'), ('mu_agg_brand_max', 'max'),('mu_agg_brand_mean', 'mean'),('mu_agg_brand_std', 'std'), ('mu_agg_brand_sum', 'sum')]).reset_index().fillna(0)
mu_buy_actions_brand_tmp.head(5)

  user_id merchant_id  mu_agg_brand
0       1        1019             1
1       1        4026             1
2       1         925             1
3      10        1364             1
4      10         938             1


,user_id,mu_agg_brand_min,mu_agg_brand_max,mu_agg_brand_mean,mu_agg_brand_std,mu_agg_brand_sum
0,1,1,1,1.0,0.0,3
1,10,1,1,1.0,0.0,2
2,100,1,1,1.0,0.0,6
3,1000,1,1,1.0,0.0,5
4,10000,1,1,1.0,0.0,13


In [38]:
matrix = matrix.merge(mu_buy_actions_brand_tmp, on=['user_id'], how='left')
del mu_buy_actions_brand_tmp,mu_buy_actions_brand
matrix

,user_id,merchant_id,label,origin,u1,u2,u3,u4,u5,u6,...,mu_agg_day_min,mu_agg_day_max,mu_agg_day_mean,mu_agg_day_std,mu_agg_day_sum,mu_agg_brand_min,mu_agg_brand_max,mu_agg_brand_mean,mu_agg_brand_std,mu_agg_brand_sum
0,34176,3906,0.0,train,451,256,45,109,108,174,...,1,6,1.545455,1.299350,34,1,1,1.0,0.0,22
1,34176,121,0.0,train,451,256,45,109,108,174,...,1,6,1.545455,1.299350,34,1,1,1.0,0.0,22
2,34176,4356,1.0,train,451,256,45,109,108,174,...,1,6,1.545455,1.299350,34,1,1,1.0,0.0,22
3,34176,2217,0.0,train,451,256,45,109,108,174,...,1,6,1.545455,1.299350,34,1,1,1.0,0.0,22
4,230784,4818,0.0,train,54,31,17,20,19,163,...,1,2,1.166667,0.408248,7,1,1,1.0,0.0,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
522336,228479,3111,nan,test,2004,1173,71,278,282,184,...,1,2,1.238095,0.436436,26,1,1,1.0,0.0,21
522337,97919,2341,nan,test,55,29,14,17,17,138,...,1,4,2.000000,1.414214,8,1,1,1.0,0.0,4
522338,97919,3971,nan,test,55,29,14,17,17,138,...,1,4,2.000000,1.414214,8,1,1,1.0,0.0,4
522339,32639,3536,nan,test,72,46,24,33,35,172,...,1,1,1.000000,0.000000,8,1,1,1.0,0.0,8


In [39]:
# Merchant aggregation features-day
mu_buy_actions_item = buy_actions.groupby(['user_id','merchant_id'], as_index=False)['item_id'].agg({'mu_agg_item': 'nunique'}).fillna(0)
print(mu_buy_actions_item.head(5))
mu_buy_actions_item_tmp = mu_buy_actions_item.groupby('user_id')['mu_agg_item'].agg([('mu_agg_item_min', 'min'), ('mu_agg_item_max', 'max'),('mu_agg_item_mean', 'mean'),('mu_agg_item_std', 'std'), ('mu_agg_item_sum', 'sum')]).reset_index().fillna(0)
mu_buy_actions_item_tmp.head(5)

  user_id merchant_id  mu_agg_item
0       1        1019            1
1       1        4026            1
2       1         925            1
3      10        1364            3
4      10         938            2


,user_id,mu_agg_item_min,mu_agg_item_max,mu_agg_item_mean,mu_agg_item_std,mu_agg_item_sum
0,1,1,1,1.000000,0.000000,3
1,10,2,3,2.500000,0.707107,5
2,100,1,4,1.500000,1.224745,9
3,1000,1,3,1.400000,0.894427,7
4,10000,1,2,1.076923,0.277350,14


In [40]:
matrix = matrix.merge(mu_buy_actions_item_tmp, on=['user_id'], how='left')
del mu_buy_actions_item_tmp,mu_buy_actions_item
matrix

,user_id,merchant_id,label,origin,u1,u2,u3,u4,u5,u6,...,mu_agg_brand_min,mu_agg_brand_max,mu_agg_brand_mean,mu_agg_brand_std,mu_agg_brand_sum,mu_agg_item_min,mu_agg_item_max,mu_agg_item_mean,mu_agg_item_std,mu_agg_item_sum
0,34176,3906,0.0,train,451,256,45,109,108,174,...,1,1,1.0,0.0,22,1,3,1.181818,0.501081,26
1,34176,121,0.0,train,451,256,45,109,108,174,...,1,1,1.0,0.0,22,1,3,1.181818,0.501081,26
2,34176,4356,1.0,train,451,256,45,109,108,174,...,1,1,1.0,0.0,22,1,3,1.181818,0.501081,26
3,34176,2217,0.0,train,451,256,45,109,108,174,...,1,1,1.0,0.0,22,1,3,1.181818,0.501081,26
4,230784,4818,0.0,train,54,31,17,20,19,163,...,1,1,1.0,0.0,6,1,2,1.166667,0.408248,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
522336,228479,3111,nan,test,2004,1173,71,278,282,184,...,1,1,1.0,0.0,21,1,2,1.190476,0.402374,25
522337,97919,2341,nan,test,55,29,14,17,17,138,...,1,1,1.0,0.0,4,1,3,1.750000,0.957427,7
522338,97919,3971,nan,test,55,29,14,17,17,138,...,1,1,1.0,0.0,4,1,3,1.750000,0.957427,7
522339,32639,3536,nan,test,72,46,24,33,35,172,...,1,1,1.0,0.0,8,1,1,1.000000,0.000000,8


# 训练集和测试集划分

In [41]:
# 分割训练数据和测试数据
matrix['user_id'] = matrix['user_id'].astype('int32')
matrix['merchant_id'] = matrix['merchant_id'].astype('int32')
train_data = matrix[matrix['origin'] == 'train'].drop(['origin'], axis=1)
test_data = matrix[matrix['origin'] == 'test'].drop(['label', 'origin'], axis=1)
train_X, train_y = train_data.drop(['label'], axis=1), train_data['label']
del temp, matrix
gc.collect()

60

# XGBoost模型训练

In [42]:
# 导入用到的模型包
import sklearn
from sklearn.model_selection import train_test_split
import xgboost as xgb
# import lightgbm as lgb
# 将训练集进行切分，20%用于验证
X_train, X_valid, y_train, y_valid = train_test_split(train_X, train_y, test_size=.2)

In [43]:
# 使用XGBoost
model = xgb.XGBClassifier(
    max_depth=8,
    n_estimators=2000,
    min_child_weight=200, 
    colsample_bytree=0.8, 
    subsample=0.8, 
    eta=0.01,    
    seed=42     
)
model.fit(
    X_train, y_train,
    eval_metric='auc', 
    eval_set=[(X_train, y_train), (X_valid, y_valid)],
    verbose=True,
    #早停法，如果auc在10epoch没有进步就stop
    early_stopping_rounds=10 
)
model.fit(X_train, y_train)

[0]	validation_0-auc:0.65685	validation_1-auc:0.63656
Multiple eval metrics have been passed: 'validation_1-auc' will be used for early stopping.

Will train until validation_1-auc hasn't improved in 10 rounds.
[1]	validation_0-auc:0.66773	validation_1-auc:0.64402
[2]	validation_0-auc:0.67241	validation_1-auc:0.64846
[3]	validation_0-auc:0.67672	validation_1-auc:0.65087
[4]	validation_0-auc:0.67747	validation_1-auc:0.65144
[5]	validation_0-auc:0.67774	validation_1-auc:0.65293
[6]	validation_0-auc:0.67836	validation_1-auc:0.65419
[7]	validation_0-auc:0.67914	validation_1-auc:0.65533
[8]	validation_0-auc:0.67950	validation_1-auc:0.65554
[9]	validation_0-auc:0.68030	validation_1-auc:0.65563
[10]	validation_0-auc:0.68030	validation_1-auc:0.65548
[11]	validation_0-auc:0.68134	validation_1-auc:0.65642
[12]	validation_0-auc:0.68169	validation_1-auc:0.65733
[13]	validation_0-auc:0.68168	validation_1-auc:0.65746
[14]	validation_0-auc:0.68161	validation_1-auc:0.65764
[15]	validation_0-auc:0.6816

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.8, eta=0.01, gamma=0,
              gpu_id=-1, importance_type='gain', interaction_constraints='',
              learning_rate=0.00999999978, max_delta_step=0, max_depth=8,
              min_child_weight=200, missing=nan, monotone_constraints='()',
              n_estimators=2000, n_jobs=0, num_parallel_tree=1, random_state=42,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=42,
              subsample=0.8, tree_method='exact', validate_parameters=1,
              verbosity=None)

 # 测试集预测

In [44]:
prob = model.predict_proba(test_data)
submission['prob'] = pd.Series(prob[:,1])
submission.drop(['origin'], axis=1, inplace=True)
submission.to_csv('prediction_i_1.csv', index=False)

# lightGBM

In [45]:
import lightgbm as lgb
# 使用LightGBM模型
model = lgb.LGBMClassifier(
    num_leaves=51,
    max_depth=10,
    boosting_type='gbdt',
    objective='binary',
    learning_rate=0.015,
    n_estimators=2000,
    subsample=0.75,
    subsample_freq=2,
    reg_lambda=0.28,
    reg_alpha=0.12,
    colsample_bytree=0.8,
    min_child_samples=300,
    min_split_gain=0.1
)
model.fit(
    X_train, y_train,
    eval_set=[(X_train, y_train), (X_valid, y_valid)],
    eval_metric='auc', 
    early_stopping_rounds=100
)

[1]	training's auc: 0.656489	training's binary_logloss: 0.229523	valid_1's auc: 0.637748	valid_1's binary_logloss: 0.230747
Training until validation scores don't improve for 100 rounds
[2]	training's auc: 0.668121	training's binary_logloss: 0.229178	valid_1's auc: 0.646575	valid_1's binary_logloss: 0.230456
[3]	training's auc: 0.672265	training's binary_logloss: 0.228809	valid_1's auc: 0.650453	valid_1's binary_logloss: 0.230162
[4]	training's auc: 0.672311	training's binary_logloss: 0.228463	valid_1's auc: 0.651005	valid_1's binary_logloss: 0.229883
[5]	training's auc: 0.673908	training's binary_logloss: 0.228138	valid_1's auc: 0.652714	valid_1's binary_logloss: 0.229616
[6]	training's auc: 0.674815	training's binary_logloss: 0.227821	valid_1's auc: 0.653881	valid_1's binary_logloss: 0.229363
[7]	training's auc: 0.676609	training's binary_logloss: 0.227506	valid_1's auc: 0.655005	valid_1's binary_logloss: 0.229111
[8]	training's auc: 0.67696	training's binary_logloss: 0.227204	valid_

LGBMClassifier(colsample_bytree=0.8, learning_rate=0.015, max_depth=10,
               min_child_samples=300, min_split_gain=0.1, n_estimators=2000,
               num_leaves=51, objective='binary', reg_alpha=0.12,
               reg_lambda=0.28, subsample=0.75, subsample_freq=2)

In [46]:
prob = model.predict_proba(test_data)
submission.drop(['prob'], axis=1, inplace=True)
submission['prob'] = pd.Series(prob[:,1])
# submission.drop(['origin'], axis=1, inplace=True)
submission.to_csv('prediction_i_2.csv', index=False)